### We try to explore some basic relationships from the trec relevant reports. We also investigate where my method does well and fails. 


Basic todo:
* parse relevance judgement score for each topic -- done
* get a document by quering from the sql database given a document ID -- done
* spot patterns in good scoring documents
* 

In [1]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="root",
  database="wapo"
)

mycursor = mydb.cursor()

mycursor.execute("SHOW TABLES")

for x in mycursor:
  print(x)

(u'reports',)


In [2]:
question_file_str = "/home/i3/git/WapoDataMining/resources/topics.xml"
output_dir_str = "/home/i3/git/WapoDataMining/results/"
answer_file_str = "/home/i3/git/WapoDataMining/bqrels.exp-gains.txt"
#rep_id = "00f57310e5c8ec7833d6756ba637332e"

In [3]:

from xml.dom import minidom
    
def getText(nodelist):
    rc = []
    for node in nodelist:
        if node.nodeType == node.TEXT_NODE:
            rc.append(node.data)
    return ''.join(rc)

def read_all_questions():
    """
     read the question xml file
     return a list of tuples
     [(topic_no,report_id),...]
    """

    newsitems = [] 
    
    xmldoc = minidom.parse(question_file_str)
    
    itemlist = xmldoc.getElementsByTagName('top')
    #print(len(itemlist))
    #print(itemlist[0].attributes['name'].value)
    for s in itemlist:
        
        #print(s.attributes['num'].value)
        docid = (getText(s.getElementsByTagName('docid')[0].childNodes))
        num = (getText(s.getElementsByTagName('num')[0].childNodes))
        num = num[9:len(num)-1]
        newsitems.append((num,docid))
        
    return newsitems
    
    
questions = read_all_questions()
#questions

"""
returns list of tuple
[(topic_id,doc_id,score), ...]
"""
def read_all_judgements():
    results = []
    f = open(answer_file_str, "r")
    for x in f:
        d = x.split()
        results.append((d[0],d[2],d[3]))
    f.close()
    return results
    
answers = read_all_judgements()
answers

[('321', '00f57310e5c8ec7833d6756ba637332e', '16'),
 ('321', '02ae7136-006d-11e3-9711-3708310f6f4d', '2'),
 ('321', '035622671d5ee6a098d686c9a4d5b18e', '16'),
 ('321', '041e530b21766fc294d4e9c48b1e2723', '0'),
 ('321', '049739753ce2e539d8dc2165daaf6aa6', '4'),
 ('321', '071068774e4eb454c1702192fcca886b', '2'),
 ('321', '071ad6f3cfba2273c7f8d5264c0a7583', '16'),
 ('321', '07578ec30a042be5678bda5f99949ff3', '0'),
 ('321', '090524c518ebedafae7ed68e16a25436', '0'),
 ('321', '09b3167f0d1aa5cfa8be932bb704d75a', '8'),
 ('321', '0a378bae-7833-11e1-b191-2d5f686e7ff4', '8'),
 ('321', '0bca80bf2a80b62a0e216540861ef7a9', '2'),
 ('321', '0caa33d172c696cac4bbc36df4e44bf2', '8'),
 ('321', '0da8ee91889db59cd8d8c4437655a0ef', '16'),
 ('321', '0e92c3bb3dbf2dc654a2a854752651ce', '8'),
 ('321', '1114e343-d128-4a7d-bb5d-66d2a9836f72', '2'),
 ('321', '12089ed7089574776db6c1e5e55704af', '8'),
 ('321', '13381e831a417a6db347ec237373d3ff', '4'),
 ('321', '139c0f1649fa758d2379555c91b9c37e', '4'),
 ('321', '13f8a

Now questions and answers contains the file contents. make them into dataframe.

In [4]:
answers_tup_df = pd.DataFrame(answers,columns=["topic_no","id","score"])
answers_tup_df

questions_tup_df = pd.DataFrame(questions,columns=["topic_no","id"])
questions_tup_df
print(answers_tup_df.shape)
print(questions_tup_df.shape)

(8508, 3)
(50, 2)


In [5]:

buf = "("
l = len(questions)

for q in questions:
    buf+="'"+q[1]+"'"
    l = l-1
    if(l!=0):
        buf+=","

buf+=")"

buf
query = "SELECT * FROM reports WHERE id IN "+buf
df_questions = pd.read_sql(query, con=mydb)
df_questions


,id,url,title,author,date,source,articleType,content
0,02e52bdba097c9df4cbae66e04f82542,https://www.washingtonpost.com/news/speaking-o...,Europe will send a rover to Mars but won’t pro...,Sarah Kaplan,1480958455000,The Washington Post,Speaking of Science,The European Space Agency just secured 436 mil...
1,0e85b0c0-f7ef-11e4-9030-b4732caefe81,https://www.washingtonpost.com/business/econom...,Meet the Mirai: Why Toyota wants to make your ...,Drew Harwell,1431473820000,The Washington Post,Business,The Toyota sedan cruising last week down a lea...
2,11915bd8-7944-11e2-9c27-fdd594ea6286,duplicate,Cuba has outsized role in Venezuela as Chavez ...,Juan Forero,1361143680000,The Washington Post,The Americas,"CARACAS, Venezuela — It’s become a well-worn r..."
3,145b9a6caa16d931c108a89798e65e17,https://www.washingtonpost.com/news/post-natio...,"How a long-lost Polish painting, likely stolen...",Elahe Izadi,1443472403000,The Washington Post,Post Nation,Polish baroque painter Krzysztof Lubieniecki f...
4,159e6f9e-8e84-11e3-84e1-27626c5ef5fb,https://www.washingtonpost.com/entertainment/m...,"Defining cool, from Walt Whitman and James Dea...",Philip Kennicott,1391648479000,The Washington Post,Museums,"Cultural concepts are always fuzzy, but that d..."
5,1e03fecf4d33b7896203298ab3858156,https://www.washingtonpost.com/news/worldviews...,Piracy is on the rise in the world’s most cruc...,Ishaan Tharoor,1412926242000,The Washington Post,WorldViews,"Last week, the Sunrise 689, a Vietnamese oil t..."
6,2a340b8573d498e261d6f2365b37f8eb,https://www.washingtonpost.com/news/animalia/w...,Teen camper wakes up to ‘crunching noise’ — an...,Cleve R. Wootson Jr.,1499692634000,The Washington Post,Animalia,"Asleep in the mountains northwest of Boulder, ..."
7,2bea9433d4e1050c9c85175df466b3e2,https://www.washingtonpost.com/news/morning-mi...,Motorized exoskeleton for paralyzed people app...,Lindsey Bever,1403859747000,The Washington Post,Morning Mix,It’s being called an exoskeleton for the paral...
8,2e83ad87eb1bade22e6e96ece616c24f,https://www.washingtonpost.com/news/capital-we...,Here’s what happened to people who tried to wa...,Angela Fritz,1498847455000,The Washington Post,Capital Weather Gang,"We all know not to stare at the sun, right? Se..."
9,30a493b8-fb07-11e4-9ef4-1bb7ce3b3fb7,https://www.washingtonpost.com/world/national-...,Boston Marathon bomber Dzhokhar Tsarnaev sente...,Adam Goldman,1431715380000,The Washington Post,National Security,BOSTON — Two years after the horrific bombing ...


In [6]:
buf = "("
l = len(answers)

for q in answers:
    buf+="'"+q[1]+"'"
    l = l-1
    if(l!=0):
        buf+=","

buf+=")"

query = "SELECT * FROM reports WHERE id IN "+buf
df_answers = pd.read_sql(query, con=mydb)
df_answers

,id,url,title,author,date,source,articleType,content
0,00041887f95b15333ae39503610f2de1,https://www.washingtonpost.com/news/wonk/wp/20...,Cholesterol in the Diet: The Long Slide from P...,Peter Whoriskey,1424378189000,The Washington Post,Wonkblog,"In one short paragraph, the new report from th..."
1,00054d36-2473-11e3-b3e9-d97fb087acd6,https://www.washingtonpost.com/local/one-injur...,One injured in Manassas hydrogen explosion at ...,Susan Svrluga; Jeremy Borden,1379959697000,The Washington Post,Local,A hydrogen tank detonated Monday and injured o...
2,000563252f7569174d0f1a2aff115854,https://www.washingtonpost.com/news/wonk/wp/20...,Greece really might leave the euro,Matt O'Brien,1422621666000,The Washington Post,Wonkblog,The world's worst portmanteau is back: Grexit....
3,000da51c-f93f-11e2-a954-358d90d5d72d,https://www.washingtonpost.com/opinions/why-po...,Why polio hasn’t been eradicated,,1375305369000,The Washington Post,Letters to the Editor,The July 30 editorial “Polio jumps the fence” ...
4,000f4c71d82856a1584c4a3c62c71454,https://www.washingtonpost.com/news/powerpost/...,Congress calls for focus on ISIS and stepped-u...,Karoun Demirjian,1458676996000,The Washington Post,PowerPost,Congressional leaders on Tuesday said the bomb...
5,0013cc4281f4c81f611bb625031ac605,https://www.washingtonpost.com/news/wonk/wp/20...,Nation’s top nutrition panel: the American die...,Peter Whoriskey,1424369012000,The Washington Post,Wonkblog,"America, please eat more fruits and vegetables..."
6,0015851c-2170-11e3-966c-9c4293c47ebe,https://www.washingtonpost.com/lifestyle/magaz...,@Gene Weingarten: A tribute to Don Graham and ...,Gene Weingarten,1381441866000,The Washington Post,Magazine,This month The Washington Post staff said a me...
7,003014d2b6c491c98fa759692e0c99da,https://www.washingtonpost.com/blogs/post-part...,An interview with New York City Mayor Bill de ...,E.J. Dionne Jr.,1398034924000,The Washington Post,PostPartisan,"NEW YORK – Last Wednesday afternoon, I intervi..."
8,0032b35853f8288d99d4f92cb5794966,https://www.washingtonpost.com/news/dr-gridloc...,"Truck spills cement on road, closes Third Stre...",Julie Zauzmer,1444255991000,The Washington Post,Gridlock,Updated at 7:12 p.m.A truck dumped sticky ceme...
9,00461b9f108425ff68f1dd910a6cf202,https://www.washingtonpost.com/news/capital-we...,Cold winters fueled ice skating boom of 19th a...,Kevin Ambrose,1452705911000,The Washington Post,Capital Weather Gang,Ice skating’s popularity boomed during the 19t...


now we have all question and answer reports from sql in df_answers and df_questions

In [7]:
print(df_answers.shape)
print(df_questions.shape)

(8275, 8)
(50, 8)


In [8]:
df_answers.describe()

,id,url,title,author,date,source,articleType,content
count,8275,8275,8275,8275,8275,8275,8275,8275
unique,8275,6961,7559,1853,7612,1,148,7639
top,5617da28-f8f9-11e6-aa1e-5f735ee31334,duplicate,What Hillary Clinton’s pneumonia diagnosis mea...,,1473674685000,The Washington Post,Wonkblog,"On Labor Day, in the middle of a Cleveland cam..."
freq,1,678,72,458,72,8275,512,71


In [9]:
# merge the corresponding df
df_merged_questions = pd.merge(left=df_questions,right=questions_tup_df, left_on='id', right_on='id')
df_merged_questions

,id,url,title,author,date,source,articleType,content,topic_no
0,02e52bdba097c9df4cbae66e04f82542,https://www.washingtonpost.com/news/speaking-o...,Europe will send a rover to Mars but won’t pro...,Sarah Kaplan,1480958455000,The Washington Post,Speaking of Science,The European Space Agency just secured 436 mil...,809
1,0e85b0c0-f7ef-11e4-9030-b4732caefe81,https://www.washingtonpost.com/business/econom...,Meet the Mirai: Why Toyota wants to make your ...,Drew Harwell,1431473820000,The Washington Post,Business,The Toyota sedan cruising last week down a lea...,375
2,11915bd8-7944-11e2-9c27-fdd594ea6286,duplicate,Cuba has outsized role in Venezuela as Chavez ...,Juan Forero,1361143680000,The Washington Post,The Americas,"CARACAS, Venezuela — It’s become a well-worn r...",807
3,145b9a6caa16d931c108a89798e65e17,https://www.washingtonpost.com/news/post-natio...,"How a long-lost Polish painting, likely stolen...",Elahe Izadi,1443472403000,The Washington Post,Post Nation,Polish baroque painter Krzysztof Lubieniecki f...,422
4,159e6f9e-8e84-11e3-84e1-27626c5ef5fb,https://www.washingtonpost.com/entertainment/m...,"Defining cool, from Walt Whitman and James Dea...",Philip Kennicott,1391648479000,The Washington Post,Museums,"Cultural concepts are always fuzzy, but that d...",433
5,1e03fecf4d33b7896203298ab3858156,https://www.washingtonpost.com/news/worldviews...,Piracy is on the rise in the world’s most cruc...,Ishaan Tharoor,1412926242000,The Washington Post,WorldViews,"Last week, the Sunrise 689, a Vietnamese oil t...",367
6,2a340b8573d498e261d6f2365b37f8eb,https://www.washingtonpost.com/news/animalia/w...,Teen camper wakes up to ‘crunching noise’ — an...,Cleve R. Wootson Jr.,1499692634000,The Washington Post,Animalia,"Asleep in the mountains northwest of Boulder, ...",336
7,2bea9433d4e1050c9c85175df466b3e2,https://www.washingtonpost.com/news/morning-mi...,Motorized exoskeleton for paralyzed people app...,Lindsey Bever,1403859747000,The Washington Post,Morning Mix,It’s being called an exoskeleton for the paral...,806
8,2e83ad87eb1bade22e6e96ece616c24f,https://www.washingtonpost.com/news/capital-we...,Here’s what happened to people who tried to wa...,Angela Fritz,1498847455000,The Washington Post,Capital Weather Gang,"We all know not to stare at the sun, right? Se...",427
9,30a493b8-fb07-11e4-9ef4-1bb7ce3b3fb7,https://www.washingtonpost.com/world/national-...,Boston Marathon bomber Dzhokhar Tsarnaev sente...,Adam Goldman,1431715380000,The Washington Post,National Security,BOSTON — Two years after the horrific bombing ...,808


In [10]:
df_merged_answers = pd.merge(left=df_answers,right=answers_tup_df, left_on='id', right_on='id')
df_merged_answers

,id,url,title,author,date,source,articleType,content,topic_no,score
0,00041887f95b15333ae39503610f2de1,https://www.washingtonpost.com/news/wonk/wp/20...,Cholesterol in the Diet: The Long Slide from P...,Peter Whoriskey,1424378189000,The Washington Post,Wonkblog,"In one short paragraph, the new report from th...",818,8
1,00054d36-2473-11e3-b3e9-d97fb087acd6,https://www.washingtonpost.com/local/one-injur...,One injured in Manassas hydrogen explosion at ...,Susan Svrluga; Jeremy Borden,1379959697000,The Washington Post,Local,A hydrogen tank detonated Monday and injured o...,375,0
2,000563252f7569174d0f1a2aff115854,https://www.washingtonpost.com/news/wonk/wp/20...,Greece really might leave the euro,Matt O'Brien,1422621666000,The Washington Post,Wonkblog,The world's worst portmanteau is back: Grexit....,378,4
3,000da51c-f93f-11e2-a954-358d90d5d72d,https://www.washingtonpost.com/opinions/why-po...,Why polio hasn’t been eradicated,,1375305369000,The Washington Post,Letters to the Editor,The July 30 editorial “Polio jumps the fence” ...,801,0
4,000f4c71d82856a1584c4a3c62c71454,https://www.washingtonpost.com/news/powerpost/...,Congress calls for focus on ISIS and stepped-u...,Karoun Demirjian,1458676996000,The Washington Post,PowerPost,Congressional leaders on Tuesday said the bomb...,341,8
5,0013cc4281f4c81f611bb625031ac605,https://www.washingtonpost.com/news/wonk/wp/20...,Nation’s top nutrition panel: the American die...,Peter Whoriskey,1424369012000,The Washington Post,Wonkblog,"America, please eat more fruits and vegetables...",818,2
6,0015851c-2170-11e3-966c-9c4293c47ebe,https://www.washingtonpost.com/lifestyle/magaz...,@Gene Weingarten: A tribute to Don Graham and ...,Gene Weingarten,1381441866000,The Washington Post,Magazine,This month The Washington Post staff said a me...,824,0
7,003014d2b6c491c98fa759692e0c99da,https://www.washingtonpost.com/blogs/post-part...,An interview with New York City Mayor Bill de ...,E.J. Dionne Jr.,1398034924000,The Washington Post,PostPartisan,"NEW YORK – Last Wednesday afternoon, I intervi...",803,0
8,0032b35853f8288d99d4f92cb5794966,https://www.washingtonpost.com/news/dr-gridloc...,"Truck spills cement on road, closes Third Stre...",Julie Zauzmer,1444255991000,The Washington Post,Gridlock,Updated at 7:12 p.m.A truck dumped sticky ceme...,363,0
9,00461b9f108425ff68f1dd910a6cf202,https://www.washingtonpost.com/news/capital-we...,Cold winters fueled ice skating boom of 19th a...,Kevin Ambrose,1452705911000,The Washington Post,Capital Weather Gang,Ice skating’s popularity boomed during the 19t...,347,0


### We are set. df_merged_questions and df_merged_answers are the only values we need to do exploration

#### Distribution of scores for all 8508 judgements

In [11]:
df_merged_answers["score"].value_counts()

0     6465
2     1189
4      584
8      164
16     106
Name: score, dtype: int64

In [12]:
a = df_merged_answers.drop_duplicates(subset="id")
a.shape

(8275, 10)

In [13]:
pd.concat([df_merged_answers, a, a]).drop_duplicates(keep=False) # ids that are duplicates

,id,url,title,author,date,source,articleType,content,topic_no,score
25,008c981a-c2e0-11e4-ad5c-3b8ce89f1b89,https://www.washingtonpost.com/national/health...,Widening superbug outbreak raises questions fo...,Brady Dennis,1425603869000,The Washington Post,Health & Science,By the time 18-year-old Aaron Young wound up a...,823,0
124,036c654e-a313-11e3-8466-d34c451760b9,https://www.washingtonpost.com/national/health...,Is organic better for your health? A look at m...,Tamar Haspel,1396907460000,The Washington Post,Health & Science,Organic or conventional? It’s a choice many gr...,823,0
147,041cba74b97a9b3c4b469cf161f83d31,https://www.washingtonpost.com/news/worldviews...,Welcome to WorldViews,Max Fisher,1350387182000,The Washington Post,WorldViews,The great tragedy of world news is that few to...,802,0
170,048e4b3282b7dca5184c2ebcc7748550,https://www.washingtonpost.com/news/wonk/wp/20...,Hillary Clinton’s top goal as president could ...,Max Ehrenfreund,1437399593000,The Washington Post,Wonkblog,"Hillary Rodham Clinton says the ""defining econ...",816,2
187,04dddf78-e728-11e6-b82f-687d6e6a3e7c,https://www.washingtonpost.com/opinions/how-ma...,How Maryland can combat the spread of antibiot...,Emily L. Heil,1486150663000,The Washington Post,Opinion,"Emily L. Heil, an assistant professor of pharm...",823,0
193,050307a650d0618078029ab90792ec69,https://www.washingtonpost.com/news/wonk/wp/20...,Here’s why Obama should get credit for reducin...,Zachary A. Goldfarb,1406563252000,The Washington Post,Wonkblog,My post last week on how President Obama has r...,816,0
240,06228d0afcf0e8d36998c31d4f29ceaa,https://www.washingtonpost.com/news/to-your-he...,FDA warns makers of medical scopes in wake of ...,Brady Dennis,1439838269000,The Washington Post,To Your Health,The Food and Drug Administration has sent warn...,823,0
249,0659042c-b3de-11e6-8616-52b15787add0,https://www.washingtonpost.com/opinions/fidel-...,Fidel Castro’s terrible legacy,Editorial Board,1480187557000,The Washington Post,The Post's View,"IN CONTRAST to his long life of violence, both...",817,0
251,0669e164-22cf-11e3-966c-9c4293c47ebe,https://www.washingtonpost.com/local/obituarie...,"Thomas E. Hutchinson, U-Va. professor, dies at 77",Matt Schudel,1379805240000,The Washington Post,Obituaries,"Thomas E. Hutchinson, a University of Virginia...",806,0
252,0669e164-22cf-11e3-966c-9c4293c47ebe,https://www.washingtonpost.com/local/obituarie...,"Thomas E. Hutchinson, U-Va. professor, dies at 77",Matt Schudel,1379805240000,The Washington Post,Obituaries,"Thomas E. Hutchinson, a University of Virginia...",350,0


In [14]:
# instance where a report occurs in two relevance judgements
df_merged_answers.loc[df_merged_answers["id"]=='048e4b3282b7dca5184c2ebcc7748550']

,id,url,title,author,date,source,articleType,content,topic_no,score
169,048e4b3282b7dca5184c2ebcc7748550,https://www.washingtonpost.com/news/wonk/wp/20...,Hillary Clinton’s top goal as president could ...,Max Ehrenfreund,1437399593000,The Washington Post,Wonkblog,"Hillary Rodham Clinton says the ""defining econ...",803,0
170,048e4b3282b7dca5184c2ebcc7748550,https://www.washingtonpost.com/news/wonk/wp/20...,Hillary Clinton’s top goal as president could ...,Max Ehrenfreund,1437399593000,The Washington Post,Wonkblog,"Hillary Rodham Clinton says the ""defining econ...",816,2


In [ ]:
df_merged_answers